In [1]:
import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = 140-96*x+12*x*x
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 100
    child_num = 50

    space = OrderedDict((
        ('x', (-2., 150.)),
        ('y', (0., 0.)),
        ('z', (0., 0.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval 55.48883493658499, 0.0, 0.0: 31761.201477522744
Eval 100.25735590533421, 0.0, 0.0: 111133.74279063415
Eval 96.22084792678116, 0.0, 0.0: 102004.21750801396
Eval 107.88635870682276, 0.0, 0.0: 129456.5063043518
Eval 105.847745585207, 0.0, 0.0: 124423.55936946861
Eval 57.79751865532086, 0.0, 0.0: 34678.076161635145
Eval 62.310216922756865, 0.0, 0.0: 40748.976770947535
Eval 96.8296761038299, 0.0, 0.0: 103356.1851865036
Eval 41.26861210668083, 0.0, 0.0: 16615.39338029884
Eval 65.73030609263796, 0.0, 0.0: 45675.5682834893
Eval 107.65025807335893, 0.0, 0.0: 128868.51198408689
Eval 71.89396116137829, 0.0, 0.0: 55263.079546192916
Eval 43.63511081445119, 0.0, 0.0: 18799.304111285903
Eval 8.77068998340112, 0.0, 0.0: 221.11379501268527
Eval 108.19059988169525, 0.0, 0.0: 130216.17324449017
Eval 126.03595588786877, 0.0, 0.0: 178661.2943535902
Eval 78.1453379513768, 0.0, 0.0: 65918.37367908652
Eval 117.00641409319032, 0.0, 0.0: 153193.39551441924
Eval 95.12748210453326, 0.0, 0.0: 99598.6159365443